In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

flow_length = 8

# dataset paths
tuesday = f"../../../datasets/Tuesday-WorkingHours_sliced_flow_{flow_length}_preprocessed.csv"
wednesday = f"../../../datasets/Wednesday-WorkingHours_sliced_flow_{flow_length}_preprocessed.csv"
thursday = f"../../../datasets/Thursday-WorkingHours_sliced_flow_{flow_length}_preprocessed.csv"
firday = f"../../../datasets/Friday-WorkingHours_sliced_flow_{flow_length}_preprocessed.csv"

output_path = f"../../dataset/distribution_duplicates_equal_{flow_length}.csv"

file_paths = [tuesday, wednesday, thursday, firday]

# initialization
original_distribution = []
original_benign_distribution = []
original_attack_distribution = []

no_duplicate_distribution = []
no_duplicate_benign_distribution = []
no_duplicate_attack_distribution = []

original_equal_distribution = []
original_equal_benign_distribution = []
original_equal_attack_distribution = []

no_duplicate_equal_distribution = []
no_duplicate_equal_benign_distribution = []
no_duplicate_equal_attack_distribution = []

for file_path in file_paths:
    df = pd.read_csv(file_path)

    # cut the duplicated flows
    # map the duplicated entires to False
    # only the first entry is marked as True
    df_dup = df.duplicated(keep='first').map({True: False, False: True})
    # extract the entries with True (the first flow entry)
    df_no_duplicate = df[df_dup]

    df_equal = df[df["bidirectional_packets"] == flow_length]
    df_no_duplicate_equal = df_no_duplicate[df_no_duplicate["bidirectional_packets"] == flow_length]

    benign_num, attack_num = np.bincount(df["Label"])
    original_distribution.append(len(df))
    original_benign_distribution.append(benign_num)
    original_attack_distribution.append(attack_num)

    benign_num, attack_num = np.bincount(df_no_duplicate["Label"])
    no_duplicate_distribution.append(len(df_no_duplicate))
    no_duplicate_benign_distribution.append(benign_num)
    no_duplicate_attack_distribution.append(attack_num)

    benign_num, attack_num = np.bincount(df_equal["Label"])
    original_equal_distribution.append(len(df_equal))
    original_equal_benign_distribution.append(benign_num)
    original_equal_attack_distribution.append(attack_num)

    benign_num, attack_num = np.bincount(df_no_duplicate_equal["Label"])
    no_duplicate_equal_distribution.append(len(df_no_duplicate_equal))
    no_duplicate_equal_benign_distribution.append(benign_num)
    no_duplicate_equal_attack_distribution.append(attack_num)

df_distribution = pd.DataFrame({
    "original_total": original_distribution,
    f"original_equal_{flow_length}_total": original_equal_distribution,
    "no_duplicate_total": no_duplicate_distribution,
    f"no_duplicate_equal_{flow_length}_total": no_duplicate_equal_distribution,
    "original_benign": original_benign_distribution,
    f"original_equal_{flow_length}_benign": original_equal_benign_distribution,
    "no_duplicate_benign": no_duplicate_benign_distribution,
    f"no_duplicate_equal_{flow_length}_benign": no_duplicate_equal_benign_distribution,
    "original_attack": original_attack_distribution,
    f"original_equal_{flow_length}_attack": original_equal_attack_distribution,
    "no_duplicate_attack": no_duplicate_attack_distribution,
    f"no_duplicate_equal_{flow_length}_attack": no_duplicate_equal_attack_distribution,
})

df_distribution.index = ["Tuesday", "Wednesday", "Thursday", "Friday"]

df_distribution.to_csv(output_path)


308349
165909
337293
185525
343134
153466
501096
202760


In [58]:
tuesday = "../../datasets/Tuesday-WorkingHours_sliced_flow_7_preprocessed.csv"

df_tues = pd.read_csv(tuesday)


In [61]:
from sklearn.utils import shuffle


df_tues = df_tues[df_tues["bidirectional_packets"] == 7]
df_tues_benign = df_tues[df_tues["Label"] == 0]
df_tues_attack = df_tues[df_tues["Label"] == 1]
attack_num = len(df_tues_attack)
# cut the replicated flow entries for benign
df_tues_benign_no_dup = df_tues_benign[df_tues_benign.duplicated(
    keep='first').map({True: False, False: True})]
# set drop=True to remove the orignal index after sampling
df_tues_benign_no_dup = df_tues_benign_no_dup.sample(attack_num).reset_index(drop=True)
# concatenate df_tues_benign_no_dup and df_tues_attack and shuffle them
df_tues = pd.concat([df_tues_benign_no_dup, df_tues_attack])
df_tues = shuffle(df_tues)

In [62]:
print(len(df_tues_benign_no_dup))
print(len(df_tues_attack))

6950
6950


In [63]:
df_tues.head(10)

,protocol,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_duration_ms,src2dst_packets,src2dst_bytes,dst2src_duration_ms,dst2src_packets,dst2src_bytes,...,splt_direction_5_0,splt_direction_5_1,splt_direction_5_2,splt_direction_6_0,splt_direction_6_1,splt_direction_6_2,splt_direction_7_0,splt_direction_7_1,splt_direction_7_2,Label
5351,6,71,7,1300,47,3,440,48,4,860,...,0,0,1,0,0,1,0,0,1,0
88477,6,90,7,34,90,4,14,90,3,20,...,0,1,0,0,1,0,0,0,1,1
1498,6,106,7,1660,52,3,200,55,4,1460,...,0,0,1,0,0,1,0,0,1,0
6288,6,406,7,680,406,4,296,203,3,384,...,0,0,1,0,0,1,0,1,0,0
279257,6,12,7,65,12,4,24,10,3,41,...,0,0,1,0,0,1,0,1,0,1
919,6,252,7,1651,252,4,191,132,3,1460,...,0,0,1,0,0,1,0,1,0,0
1158,6,67,7,536,67,5,426,29,2,110,...,0,0,1,0,1,0,0,1,0,0
1278,6,36,7,3126,16,3,206,20,4,2920,...,0,0,1,0,0,1,0,0,1,0
3534,6,481,7,285,481,5,147,239,2,138,...,0,0,1,0,1,0,0,1,0,0
182821,6,11,7,65,11,4,24,10,3,41,...,0,0,1,0,0,1,0,1,0,1
